In [35]:
import findspark
findspark.init()

In [36]:
from pyspark.sql import SparkSession
import time

spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [37]:
# 1. Read from the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
homes_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
homes_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [38]:
# 2. Create a temporary view of the DataFrame
homes_df.createOrReplaceTempView('home_sales')

In [39]:
# 3. Finding the average price for a four bedroom house sold in each year rounded to two decimal places
avg_price_4BR = """
SELECT YEAR(date) as YEAR, ROUND(AVG(price), 2) AS AVERAGE_PRICE FROM home_sales 
where bedrooms = 4 GROUP BY YEAR ORDER BY YEAR DESC"""

spark.sql(avg_price_4BR).show()

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [40]:
# 4. Finding the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places

avg_price_3a = """
SELECT YEAR(date_built) AS YEAR, ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3 and bathrooms = 3 GROUP BY YEAR(date_built) ORDER BY YEAR DESC"""

spark.sql(avg_price_3a).show()


+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2017|    292676.79|
|2016|    290555.07|
|2015|     288770.3|
|2014|    290852.27|
|2013|    295962.27|
|2012|    293683.19|
|2011|    291117.47|
|2010|    292859.62|
+----+-------------+



In [41]:
#  5.Finding the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors, and are greater than or equal to 2,000 square feet rounded to two decimal places

avg_price_3b = """
SELECT YEAR(date_built) AS YEAR_BUILT, ROUND(AVG(price), 2) AS AVERAGE_PRICE FROM home_sales 
WHERE bedrooms = 3 and bathrooms = 3  and sqft_living >= 2000 and floors = 2 
GROUP BY YEAR_BUILT ORDER BY YEAR_BUILT DESC"""

spark.sql(avg_price_3b).show()

+----------+-------------+
|YEAR_BUILT|AVERAGE_PRICE|
+----------+-------------+
|      2017|    280317.58|
|      2016|     293965.1|
|      2015|    297609.97|
|      2014|    298264.72|
|      2013|    303676.79|
|      2012|    307539.97|
|      2011|    276553.81|
|      2010|    285010.22|
+----------+-------------+



In [42]:
# 6. Finding the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than or equal to $350,000
# and  determinining the run time for this query.

start_time = time.time()
view_rating="""
SELECT view, ROUND(AVG(price), 2) AS AVERAGE_PRICE FROM home_sales 
GROUP BY view HAVING AVG(price) >= 350000 ORDER BY view desc"""

spark.sql(view_rating).show()

print("Time taken is: %s seconds" %(time.time() - start_time))


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

Time taken is: 0.2576727867126465 seconds


In [43]:
# 7. Cache the the temporary table home_sales.

spark.sql('CACHE TABLE home_sales')

DataFrame[]

In [44]:
# 8. Checking if the table is cached.

spark.catalog.isCached('home_sales')


True

In [45]:
# 9. Running the query using the cached data,  that filters out the view ratings with average price
#  greater than or equal to $350,000 and  the runtime and compare it to uncached runtime.

start_time = time.time()
view_rating="""
SELECT view, ROUND(AVG(price), 2) AS AVERAGE_PRICE FROM home_sales 
GROUP BY view HAVING AVG(price) >= 350000 ORDER BY view desc"""

spark.sql(view_rating).show()

print("Time taken with data cached is: %s seconds" %(time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

Time taken with data cached is: 0.1631767749786377 seconds


In [46]:
# 10. Partitioning by the "date_built" field on the formatted parquet home sales data

homes_df.write.partitionBy('date_built').parquet('parquet_home_sales', mode = 'overwrite')

In [47]:
# 11. Reading the formatted parquet data

parquet_df = spark.read.parquet('parquet_home_sales')

In [48]:
spark.sql('UNCACHE TABLE home_sales')

DataFrame[]

In [54]:
# 12. Runing the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame, Rounding the average to two decimal places.
# Determining the runtime and compare it to the cached version.

start_time = time.time()
avg_price_350parquet = """
SELECT view, ROUND(AVG(price), 2) AS AVERAGE_PRICE FROM parquet_df 
GROUP BY view HAVING AVG(price) >= 350000 ORDER BY view desc"""

print("Time taken with parquet data is: %s seconds" %(time.time() - start_time))

#0.00008249282 seconds

Time taken with parquet data is: 3.814697265625e-05 seconds


In [50]:
# Checking if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
  print('home_sales is cached')
else:
  print('home_sales is not cached')

home_sales is not cached


In [53]:
#code snippet that demonstrates how to fetch executor information from the Spark REST API.

import requests
import json

def fetch_executor_details():
    # URL for the Spark REST API endpoint for executors
    spark_rest_api_url = 'http://localhost:4040/api/v1/applications'
    
    # Fetch list of applications
    apps_response = requests.get(spark_rest_api_url)
    apps = apps_response.json()
    
    if not apps:
        print("No applications found.")
        return
    
    # details from the first application
    app_id = apps[0]['id']
    
    # Construct URL to fetch executor details for the application
    executors_url = f'{spark_rest_api_url}/{app_id}/executors'
    
    # Fetch executor details
    executors_response = requests.get(executors_url)
    executors = executors_response.json()
    
    # Pretty print the executor details
    print(json.dumps(executors, indent=4))

# Call the function to fetch and print executor details
fetch_executor_details()

[
    {
        "id": "driver",
        "hostPort": "172.25.195.172:36539",
        "isActive": true,
        "rddBlocks": 0,
        "memoryUsed": 73071,
        "diskUsed": 0,
        "totalCores": 8,
        "maxTasks": 8,
        "activeTasks": 0,
        "failedTasks": 0,
        "completedTasks": 17,
        "totalTasks": 17,
        "totalDuration": 133335,
        "totalGCTime": 971,
        "totalInputBytes": 21361922,
        "totalShuffleRead": 14987,
        "totalShuffleWrite": 14987,
        "isBlacklisted": false,
        "maxMemory": 455501414,
        "addTime": "2024-02-27T03:05:14.472GMT",
        "executorLogs": {},
        "memoryMetrics": {
            "usedOnHeapStorageMemory": 73071,
            "usedOffHeapStorageMemory": 0,
            "totalOnHeapStorageMemory": 455501414,
            "totalOffHeapStorageMemory": 0
        },
        "blacklistedInStages": [],
        "peakMemoryMetrics": {
            "JVMHeapMemory": 184626496,
            "JVMOffHeapMemory

In [34]:
spark.stop()